In [1]:
import os
os.chdir("../")

In [55]:
import pandas as pd
import numpy as np
from livability_score import livability

In [3]:
df = pd.read_csv('./data/chicago_pollutants_by_latlong.csv')

In [4]:
df.head()

,weekend,hod,lat,long,PM 2.5,PM 10,Ozone,Nitrogen dioxide,Sufur dioxide,Carbon monoxide,Hydrogen Sulfide,City,Zip Code
0,0,0,41.648636,-87.526155,0.959344,2.611243,0.058071,0.016962,0.027167,1.087085,0.008230,Chicago,60633
1,0,1,41.648636,-87.526155,1.993045,2.448509,0.054879,0.018229,0.029572,1.036201,0.010529,Chicago,60633
2,0,2,41.648636,-87.526155,1.567117,2.522785,0.054829,0.017193,0.022410,0.990954,0.005744,Chicago,60633
3,0,3,41.648636,-87.526155,1.426599,2.834606,0.052170,0.017659,0.019370,0.956174,0.005080,Chicago,60633
4,0,4,41.648636,-87.526155,1.348170,2.911302,0.050052,0.016896,0.022910,0.921395,0.016872,Chicago,60633


In [5]:
def df_livability(row):
    return livability(row['Zip Code'], row['PM 2.5'], row['PM 10'], row['Ozone'], row['Nitrogen dioxide'], row['Sufur dioxide'])
# lambda row: livability(row['Zip Code'], row['PM 2.5'], row['PM 10'], row['Ozone'], row['Nitrogen dioxide'], row['Sufur dioxide'])

In [6]:
df['Livability'] = df.apply(df_livability, axis=1)

In [7]:
df.head()

,weekend,hod,lat,long,PM 2.5,PM 10,Ozone,Nitrogen dioxide,Sufur dioxide,Carbon monoxide,Hydrogen Sulfide,City,Zip Code,Livability
0,0,0,41.648636,-87.526155,0.959344,2.611243,0.058071,0.016962,0.027167,1.087085,0.008230,Chicago,60633,-1.0
1,0,1,41.648636,-87.526155,1.993045,2.448509,0.054879,0.018229,0.029572,1.036201,0.010529,Chicago,60633,-1.0
2,0,2,41.648636,-87.526155,1.567117,2.522785,0.054829,0.017193,0.022410,0.990954,0.005744,Chicago,60633,-1.0
3,0,3,41.648636,-87.526155,1.426599,2.834606,0.052170,0.017659,0.019370,0.956174,0.005080,Chicago,60633,-1.0
4,0,4,41.648636,-87.526155,1.348170,2.911302,0.050052,0.016896,0.022910,0.921395,0.016872,Chicago,60633,-1.0


In [71]:
output = pd.read_csv('./data/output.csv')

In [72]:
len(output)

480000

In [12]:
idx = output.index[0]

In [13]:
row = output.loc[idx]

In [20]:
len(df[(df.lat == row['lat']) & (df.long == row['long'])])
# df[(df.lat == row['lat'])]
# df[(df.long == row['long'])]

0

In [21]:
def df_lat_long_livability(row):
    results = df[(df.lat == row['lat']) & (df.long == row['long'])]
    if len(results) > 0:
        return results.iloc[0].loc['Livability']
    return None

In [22]:
output['Livability'] = output.apply(df_lat_long_livability, axis=1)

In [29]:
dfLatLonZip = df[['lat','long','Zip Code']].copy().drop_duplicates()

In [48]:
dfLatLonZip.set_index(['lat','long'], inplace=True)

In [54]:
dfLatLonLiv = df[['lat','long','Livability']].copy().drop_duplicates().set_index(['lat','long'])

In [57]:
def df_lat_lon_livability(row):
    exists = np.any(dfLatLonLiv.index.isin([(row['lat'], row['long'])]))
    if not exists:
        return None
    return dfLatLonLiv.loc[(row['lat'], row['long']), 'Livability']

In [58]:
output['Livability'] = output.apply(df_lat_lon_livability, axis=1)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1367: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


KeyboardInterrupt: 

In [62]:
output = output.drop('Livability',axis=1)
output.head()

,weekend,hod,lat,long,PM 2.5,PM 10,Ozone,Nitrogen dioxide,Sufur dioxide,Carbon monoxide,Hydrogen Sulfide
0,0.0,0.0,41.644818,-87.526155,0.959344,2.611243,0.058071,0.016962,0.027167,1.086456,0.008230
1,0.0,1.0,41.644818,-87.526155,1.993045,2.448509,0.054879,0.018229,0.029572,1.035572,0.010529
2,0.0,2.0,41.644818,-87.526155,1.567117,2.522785,0.054829,0.017193,0.022410,0.990371,0.005744
3,0.0,3.0,41.644818,-87.526155,1.426599,2.834606,0.052170,0.017659,0.019370,0.955591,0.005080
4,0.0,4.0,41.644818,-87.526155,1.348170,2.911302,0.050052,0.016896,0.022910,0.920811,0.016872


In [89]:
dfJoined = output.copy()
dfJoined['Livability'] = dfJoined.apply(df_lat_lon_livability, axis=1)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1367: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


KeyboardInterrupt: 

In [87]:
dfJoined = output.join(dfLatLonLiv, on=('lat','long'), how='left')

In [88]:
len(output)

480000

In [85]:
len(dfJoined.loc[0:480000-1])

482256

In [79]:
len(dfJoined[output.index])

KeyError: "Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,\n                 8,      9,\n            ...\n            479990, 479991, 479992, 479993, 479994, 479995, 479996, 479997,\n            479998, 479999],\n           dtype='int64', length=480000) not in index"

In [70]:
len(dfJoined.drop_duplicates())

482256

In [68]:
dfJoined.to_csv('./data/final.csv', index=False)